## Predspracovanie dát a ETL

In [1]:
# dependenciesimport pandas as pd
import numpy as np
import pandas as pd

In [2]:
# local modules
import sys
sys.path.append("..")

from _ import constants
from _.functions import drop

In [3]:
# drop redundant data if not exists
if not constants.TRAIN_DROPPED.exists():
    df_train = drop(constants.TRAIN);
    df_train.to_parquet(constants.TRAIN_DROPPED, index=False)
else:
    df_train = pd.read_parquet(constants.TRAIN_DROPPED)
    
if not constants.TEST_DROPPED.exists():
    df_test = drop(constants.TEST);
    df_test.to_parquet(constants.TEST_DROPPED, index=False)
else:
    df_test = pd.read_parquet(constants.TEST_DROPPED)

drop_single_action_users: Dropped 56386 records.
drop_single_and_excessive_step_sessions: Dropped 1159785 records.
drop_duplicate_steps_in_session: Dropped 7 records.
test.csv - Previously 3782335, now 2566157 (Dropped 1216178 in total).


In [4]:
df_train.head(10)

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,None,None,None
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,None,None,None
2,00RL8Z82B2Z1,aff3928535f48,1541037522,3,interaction item image,666856,AU,"Sydney, Australia",mobile,None,None,None
3,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,None,None,None
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,None,None,None
5,00RL8Z82B2Z1,aff3928535f48,1541037532,6,interaction item image,666856,AU,"Sydney, Australia",mobile,None,None,None
6,00RL8Z82B2Z1,aff3928535f48,1541037532,7,interaction item image,109038,AU,"Sydney, Australia",mobile,None,None,None
7,00RL8Z82B2Z1,aff3928535f48,1541037532,8,interaction item image,666856,AU,"Sydney, Australia",mobile,None,None,None
8,00RL8Z82B2Z1,aff3928535f48,1541037542,9,interaction item image,109038,AU,"Sydney, Australia",mobile,None,None,None
9,00RL8Z82B2Z1,aff3928535f48,1541037542,10,interaction item image,109038,AU,"Sydney, Australia",mobile,None,None,None


### Úprava multi-value atribútov

Rozdelenie "|" separovaných hodnôt na list.

In [5]:
df_train['impressions'] = df_train['impressions'].apply(lambda x: x.split('|') if x != None else x)

In [6]:
df_train['prices'] = df_train['prices'].apply(lambda x: x.split('|') if x != None else x)

In [7]:
df_train['current_filters'] = df_train['current_filters'].apply(lambda x: x.split('|') if x != None else x)

## Enkódovanie 

### Enkódovanie kategorických atribútov

In [8]:
categorical_attributes = ['user_id','session_id', 'action_type', 'platform', 'city']

In [9]:
for cat in categorical_attributes:
    df_train[cat] = df_train[cat].astype('category')
    df_train[cat] = df_train[cat].cat.codes

In [10]:
df_train.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,478,469138,1541037460,1,9,Newtown,4,26837,mobile,None,None,None
1,478,469138,1541037522,2,4,666856,4,26837,mobile,None,None,None
2,478,469138,1541037522,3,4,666856,4,26837,mobile,None,None,None
3,478,469138,1541037532,4,4,666856,4,26837,mobile,None,None,None
4,478,469138,1541037532,5,4,109038,4,26837,mobile,None,None,None


### One-hot enkódovanie

Zamýšlali sme sa nad one-hot enkódovaním aj atribútu platform, ale kedže nie je z nášho pohľadu až tak dôležitý pre predikciu a obsahuje 55 rozličných hodnôt, rozhodli sme sa len pre city.

In [11]:
df_train = pd.get_dummies(df_train, columns=["device"])

In [12]:
df_train.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,current_filters,impressions,prices,device_desktop,device_mobile,device_tablet
0,478,469138,1541037460,1,9,Newtown,4,26837,None,None,None,0,1,0
1,478,469138,1541037522,2,4,666856,4,26837,None,None,None,0,1,0
2,478,469138,1541037522,3,4,666856,4,26837,None,None,None,0,1,0
3,478,469138,1541037532,4,4,666856,4,26837,None,None,None,0,1,0
4,478,469138,1541037532,5,4,109038,4,26837,None,None,None,0,1,0


## Výstup

Zoradenie podla stepu v každej session a zahodenie atribútu step

In [13]:
df_train.sort_values(['session_id','step'])
# no longer needed column
del df_train['step']

In [14]:
df_train.to_parquet(constants.TRAIN_PREPROCESSED, index=False)